<center><img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQXnrruP7guibF61c4gOESBR2VhhqYmEjKin0yY9lrhCak3hIbXI0QDSlObO74xcBkjMw&usqp=CAU"/></center>

# 🔍 Build a nest - OFFLINE 

<small>
This notebook allows you to build a child grid/a nest from a parent grid (created with <strong>`make_grid.ipynb`</strong>) with offline or AGRIF methods
<br>💡 It is linked to the croco-ocean/croco_pytools' <a href= "https://gitlab.inria.fr/croco-ocean/croco_pytools/-/issues/4">ISSUE #4</a> located on the <strong>croco-ocean INRIA's gitlab</strong>
<br>➡️ Notebook created by <a href="mailto:thibault.delahaye@ird.fr">Thibault Delahaye</a> and based on the existing codes of pycrocotool
<br>🌳 Developped on **notebook-grid-tools**'s branch, whose source is the main branch
</small>

## Table of contents 🔗

1. [Code presentation 📖](#📖-Code-presentation)
   - 
   - 

 

## 📖-Code-presentation

### Method

<div class="alert alert-block alert-info">

To represent fine-scale phenomena, CROCO offers the ability to create zooms with increased resolution. Two options exist 🛠️:

<div style="border: 2px dotted black; padding: 10px; margin: 10px;">
    
##### Offline Zoom 🗺️ ➡️ 🔍

- **Description**: Creation of a grid with the desired resolution, where the boundaries are within a grid of a larger CROCO domain, already modeled.
- **Conditions**: The high-resolution grid takes as initial and boundary conditions those from the larger CROCO simulation. The simulations are independent and no feedback from the high-resolution grid to the coarser grid is possible. The simulations run separately.
- **Advantages**:
  - Independence
  - No constraints on the chosen resolution
- **Disadvantages**:
  - No feedback to the larger domain
  - Zoom boundary forcing depends on the frequency of the parent grid’s history, which may filter out some waves present in the parent grid.
</div>
##### AGRIF Zoom 🔄

- **Description**: AGRIF is a library that allows running CROCO with several embedded domains together. (see documentation in CROCO documentation)
- **Conditions**: Parent and child(s) domains are run simultaneously, coupling the domains at the barotropic time step.
- **Advantages**:
  - Two-way feedback to the parent domain is possible and can significantly improve results of the larger domain
  - Coupling at the highest possible frequency
  - Consistency of numerics
  - Any number of nesting levels is allowed
- **Disadvantages**:
  - Requires simultaneous execution of the parent and child(s) grids
  - Nesting ratio should be 3 or 5
  - Currently, AGRIF zooms in CROCO must have the same vertical resolution as the parent.

</div>

### 💾 Datasets

Further Information: [CROCO OCEAN WEBSITE](http://www.croco-ocean.org)
  
*This file is part of CROCOTOOLS* 🐊

**Create a CROCO grid file** 🌐   
In the current state the script can handle:  

<table style="width: 100%; text-align: left; border-collapse: collapse;">
  <thead>
    <tr>
      <th style="border: 1px solid #ddd; padding: 8px; background-color: #f4f4f4;">DATA</th>
      <th style="border: 1px solid #ddd; padding: 8px; background-color: #f4f4f4;">SOURCE</th>
      <th style="border: 1px solid #ddd; padding: 8px; background-color: #f4f4f4;">FILE EXTENSION</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td style="border: 1px solid #ddd; padding: 8px;"><strong>⛰️ topo/bathy</strong></td>
      <td style="border: 1px solid #ddd; padding: 8px;">etopo (5,2,1),<br>srtm30, gebco</td>
      <td style="border: 1px solid #ddd; padding: 8px;"><em>NetCDF</em></td>
    </tr>
    <tr>
      <td style="border: 1px solid #ddd; padding: 8px;"><strong>🏖️ coastline</strong></td>
      <td style="border: 1px solid #ddd; padding: 8px;">GSHHS</td>
      <td style="border: 1px solid #ddd; padding: 8px;"><em>shapefile</em></td>
    </tr>
    <tr>
      <td style="border: 1px solid #ddd; padding: 8px;"><strong>🐊 parent CROCO grid</strong></td>
      <td style="border: 1px solid #ddd; padding: 8px;">croco_grd built with make_grid.ipynb</td>
      <td style="border: 1px solid #ddd; padding: 8px;"><em>NetCDF</em></td>
    </tr>
  </tbody>
</table>

### ⚙️ Requirements
➡️ Installing `ipympl` for interactive Plots :

*To use **interactive plotting** features with the `%matplotlib widget` [magic command](#Magic-cells) in Jupyter Notebook or JupyterLab, you need to install the `ipympl` package. This package enables Matplotlib to render interactive plots using Jupyter widgets.*

 **Run the following command** in a code cell or in your notebook to install `ipympl`:
   ```python
   pip install ipympl
   ```

---

<div style="text-align: center; font-size: 0.9em; color: gray;">
  CODE BODY
</div>

---

### Magic cells

[Magic cell](https://ipython.readthedocs.io/en/stable/interactive/magics.html) ⚡🐚 dedicated to the activation of backend environments

In [10]:
#Allows widget interactivity with user
%matplotlib widget    
%load_ext autoreload
%autoreload 2

### Dependencies

In [11]:
import numpy as np
import os,sys
os.environ['ETS_TOOLKIT'] = 'wx'
sys.path.append("./Modules/")
sys.path.append("./Readers/")
sys.path.append("./Modules/graphicUI_tools/")
from main_window import *
import tools_make_grid
#----------------------------------------------#
from tools_make_grid import inputs,inputs_smth,EasyGrid,GetMask,GetTopo
from croco_class import CROCO
from main_window import Outputs
#----------------------------------------------#
from Modules.map_tools.map_tools import plot_grid, plot_outline, plot_topo
from scipy.spatial import distance
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact_manual
#----------------------------------------------#
from matplotlib.widgets import RectangleSelector
from ipywidgets import FloatText, VBox
from IPython.display import display

## 🗺️ Parent grid path and infos

### Parent grid path:

In [31]:
parent_grid_file= "../../CROCO_FILES/croco_grd.nc"

### Collecting parent's grid inputs parameters

In [26]:
import json
import pandas as pd

output_folder = os.path.dirname(parent_grid_file)
json_filename = 'user_inputs.json'

json_file_path= os.path.join(output_folder, json_filename)

# Lire le fichier JSON
with open(json_file_path, 'r') as json_file:
    user_inputs_parent = json.load(json_file)
    
# Convertir les sous-dictionnaires en DataFrames séparés
df_inputs = pd.DataFrame([user_inputs_parent['inputs']])
df_inputs_smth = pd.DataFrame([user_inputs_parent['inputs_smth']])

**PARENT GRID INPUTS**

In [27]:
df_inputs

,tra_lon,tra_lat,size_x,size_y,rot,nx,ny
0,15,-32,1556,1334,0,39,40


**PARENT GRID SMOOTHING PARAMETERS**

In [28]:
df_inputs_smth

,depthmin,depthmax,smthr,rfact,smooth
0,50,6000,2,0.2,lsmooth


## 🐣 Build the nest

### Create a new grid

#### ✏️🔧 User's changes

##### Center's position (degrees) 📍

In [1]:
tra_lon =  15 # Longitude of the grid center 
tra_lat = -32 # Latitude of the grid center

##### Grid extent (km) and resolution

In [14]:
# Grid size [km]
size_x = 150
size_y = 130

# Grid number of points 
nx = 39
ny = 40

In [15]:
# Calculate Resolution
delta_x = size_x / nx
delta_y = size_y / ny

# Display results
print(f"Resolution in X direction: {delta_x:.2f} km")
print(f"Resolution in Y direction: {delta_y:.2f} km")

Resolution in X direction: 3.85 km
Resolution in Y direction: 3.25 km


##### Grid rotation (degrees)

In [16]:
rot = 0

#### Smoothing settings

See [online documentation](https://croco-ocean.gitlabpages.inria.fr/croco_doc/model/model.grid.html) for more details 📚

In [17]:
hmin        = 50    # Minimum depth [m]
hmax        = 6000  # Maximum depth [m]
interp_rad  = 2     # Interpolation radius in number of points (usually between 2 and 8)
rfact       = 0.2   # Maximum r-fact to reach (the lower it is, the smoother it will be)    
smooth_meth = 'lsmooth' # Smoothing method ('smooth', 'lsmooth', 'lsmooth_legacy', 'lsmooth2', 'lsmooth1', 'cond_rx0_topo') 

##### Paths

###### Inputs

Topo/bathy file ⛰️ :

In [18]:
topofile = '../../DATASETS_CROCOTOOLS/Topo/etopo2.nc'

Coastline file (for the mask) 🏖️ :

In [19]:
shp_file = '../../DATASETS_CROCOTOOLS/gshhs/GSHHS_shp/i/GSHHS_i_L1.shp'

###### Output(s)

Output grid file 🔲:

In [20]:
output_file="../../CROCO_FILES/croco_grd.nc"


##### Mask

Single Connect (Masks water not connected to the main water body) 🎭:

In [21]:
sgl_connect=[False,20,20] # True or False, point indices inside the main water body